In [17]:
from bs4 import BeautifulSoup
import pymongo
import requests
import re
import sys

In [18]:
# Create mongodb connection
from pymongo import MongoClient
connection = MongoClient('localhost', 27017)
database = connection.housing_data

table = database.etuovi

#table.drop() to drop the collection 'housing_data'

cookie = {'org.springframework.mobile.device.site.CookieSitePreferenceRepository.SITE_PREFERENCE':'NORMAL','JSESSIONID':'183EE2F12C128B603790A0D5D1558022','evid_0006':'87ccc59d-4b80-489f-98cf-be92d6591fa6','evid_0006_ref':'false','adptseg_0006':'kv1001#a-kv1002#c-kv1003#d-kv1004#d-kv1005#c-kv1006#a:b-kv1008#c-kv1009#17-kv1010#b-kv1011#f-kv1012#c-kv1013#a-kv1016#1:11:13:2:7-kv1017#2-','adptset_0006':'1','evid_0006_set':'2','__utma':'82691985.304271103.1385626909.1385642039.1385658686.4','__utmb':'82691985.9.8.1385658723878','__utmc':'82691985','__utmz':'82691985.1385626909.1.1.utmcsr=kuluttaja.etuovi.com|utmccn=(referral)|utmcmd=referral|utmcct=/crometapp/product/realties/common/public/frontpage2/search_in_progress_to_vuokraovi.jsp','slbp_80':'2265258762.20480.0000','slbp_443':'2265258762.47873.0000'}

data_added = 0

In [19]:
start_page = 1
end_page = 10
Links = []
for page in range(start_page, end_page):
    print ('Parsing page:', str(page))
    html = requests.post('http://www.vuokraovi.com/vuokra-asunnot?page=' + str(page), cookies=cookie, data={'listorder':11})
    
    parsed_html = BeautifulSoup(html.text,'html.parser')     
    #print(parsed_html.prettify()) 
    url_tag = parsed_html.find_all('a', {'class':'list-item-link'})
    for link in url_tag:
        address = link.get('href')
        if '/vuokra-asunto' in address:
            #print(address)
            Links.append(address)

print (Links)

Parsing page: 1
Parsing page: 2
Parsing page: 3
Parsing page: 4
Parsing page: 5
Parsing page: 6
Parsing page: 7
Parsing page: 8
Parsing page: 9
['/vuokra-asunto/jarvenpaa/poytaalho/kerrostalo/663227?entryPoint=fromSearch&rentalIndex=1', '/vuokra-asunto/kuopio/petonen/kerrostalo/471836?entryPoint=fromSearch&rentalIndex=2', '/vuokra-asunto/kuopio/puijonlaakso/kerrostalo/663228?entryPoint=fromSearch&rentalIndex=3', '/vuokra-asunto/jyvaskyla/palokka/kerrostalo/663226?entryPoint=fromSearch&rentalIndex=4', '/vuokra-asunto/nokia/keskusta/kerrostalo/643933?entryPoint=fromSearch&rentalIndex=5', '/vuokra-asunto/oulu/linnanmaa/kerrostalo/663221?entryPoint=fromSearch&rentalIndex=6', '/vuokra-asunto/helsinki/etela-haaga/kerrostalo/663224?entryPoint=fromSearch&rentalIndex=7', '/vuokra-asunto/tampere/kaleva/kerrostalo/663223?entryPoint=fromSearch&rentalIndex=8', '/vuokra-asunto/kemi/rytikari/kerrostalo/663222?entryPoint=fromSearch&rentalIndex=9', '/vuokra-asunto/helsinki/verajalaakso/kerrostalo/56212

In [21]:
allApartments = []
apartment = dict()
for page in Links:
    #print(page)
    html = requests.post('http://www.vuokraovi.com' + page, cookies=cookie, data={'listorder':11})
    parsed_html = BeautifulSoup(html.text,'html.parser') 
    #print(parsed_html)
    apartment['renter'] = parsed_html.find('b').get_text(strip=True)
    test_tagth = parsed_html.find_all('th')
    test_tagtd = parsed_html.find_all('td')
    apartment['postal_code'] = test_tagtd[0].find('span', {'itemprop':'addressLocality'}).get_text(strip = True)[:5]
  
    
    for i in range(0,len(test_tagth)):
        if 'Vakuus:' in test_tagth[i].get_text():
            apartment['deposit'] = test_tagtd[i].get_text(strip = True)
            
        if 'Vuokra:' in test_tagth[i].get_text():
            apartment['rent'] = test_tagtd[i].get_text(strip = True) 
            
        if 'Tyyppi:' in test_tagth[i].get_text():
            apartment['type'] = test_tagtd[i].get_text(strip = True)
            
        if 'Huoneiden lukumäärä' in test_tagth[i].get_text():
            apartment['number_of_rooms'] = test_tagtd[i].get_text(strip = True)
            
        if 'Rakennusvuosi:' in test_tagth[i].get_text():
            apartment['built_year'] = test_tagtd[i].get_text(strip = True)
            
        if 'Yleiskunto:' in test_tagth[i].get_text():
            apartment['condition'] = test_tagtd[i].get_text(strip = True)
            
        if 'sallittu:' in test_tagth[i].get_text():
            apartment['allow_pets'] = test_tagtd[i].get_text(strip = True)
            
        if 'pinta-ala:' in test_tagth[i].get_text():
            apartment['total_size'] = test_tagtd[i].get_text(strip = True)
     
    try:
        # Push the collected data to db.
        table.insert_one({
                'renter':apartment['renter'],
                'postal_code': apartment['postal_code'],
                'built_year': apartment['built_year'], 
                'type': apartment['type'],
                'total_size': apartment['total_size'],
                'number_of_rooms': apartment['number_of_rooms'],
                'condition': apartment['condition'],
                'rent': apartment['rent'],
                'deposit': apartment['deposit'],
                'allow_pets': apartment['allow_pets']          
            }) 
        data_added += 1
    except pymongo.errors.DuplicateKeyError:
            pass
    
    
    
    #To check in mongodb on your laptop, install mongodb -> sudo service mongod start-> type mongo to open mongodb console -
    # then 'use housing_data' -> db.etuovi.count() -- number of entries, db.etuovi.find() --check entries
    
    
    # At the moment, for 9 pages, db.etuovi.count() = 181 i.e. 181 entries
    #please ignore the id 

In [31]:
import pandas as pd
df = pd.read_json("apartment_data.json", lines=True)
df

,_id,allow_pets,built_year,condition,deposit,number_of_rooms,postal_code,rent,renter,total_size,type
0,{'$oid': '59e869611a98041246b41867'},ei sallittu,1997,hyvä,250 €,Kaksio,4410,719 €/kk,Lumo-kotikeskus PKS,55 m²,kerrostalo
1,{'$oid': '59e869611a98041246b41868'},ei sallittu,1989,hyvä,250 €,Kaksio,70820,630 €/kk,Lumo-kotikeskus Kuopio / VVO,45 m²,kerrostalo
2,{'$oid': '59e869631a98041246b41869'},ei sallittu,1982,hyvä,"630,00 e",Kaksio,70200,630 €/kk,OVV Asuntopalvelut Kuopio,57 m²,kerrostalo
3,{'$oid': '59e869631a98041246b4186a'},ei sallittu,2013,hyvä,600 €,Yksiö,40270,"470,81 €/kk","Lakea Oy, Seinäjoki","36,5 m²",kerrostalo
4,{'$oid': '59e869641a98041246b4186b'},ei sallittu,1949,tyydyttävä,780 €,Yksiö,37100,390 €/kk,Kiinteistövälitys Rooma Oy LKV,28 m²,kerrostalo
5,{'$oid': '59e869641a98041246b4186c'},ei sallittu,2017,hyvä,525,Yksiö,90570,525 €/kk,Lauri Kytölä,"32,5 m²",kerrostalo
6,{'$oid': '59e869641a98041246b4186d'},ei sallittu,1956,hyvä,1400.00,Yksiö,320,700 €/kk,Kiinteistötoimisto Takio LKV [A],"24,5 m²",kerrostalo
7,{'$oid': '59e869651a98041246b4186e'},ei sallittu,1943,ei luokiteltu,1200.00,Yksiö,33540,600 €/kk,Asuntolähde LKV,"37,5 m²",kerrostalo
8,{'$oid': '59e869651a98041246b4186f'},ei sallittu,1955,hyvä,450 €,3 huonetta,94830,450 €/kk,Kiinteistönvälitys Alagroup Oy LKV,50 m²,kerrostalo
9,{'$oid': '59e869661a98041246b41870'},ei sallittu,1988,hyvä,450 €,Kaksio,650,929 €/kk,SATO Vuokraus / Helsinki,53 m²,kerrostalo


In [32]:
df.to_csv('apartment.csv')